# KOL Ascension Log Wrapper (v1)
_A Styling Project by Aaron M. (AKA Captain Scotch, #437479)_

Hey all! This is a wrapper meant to isolate and analyze Ascension logs via Python, formatting everything into a clean Excel wrapper. Originally, this was simply an overlay of [CKB's run log parser in ASH](https://kolmafia.us/showthread.php?22963-RunLogSummary), which is fantastic, but as I'm more of a Python dev this quickly enlarged into a broader project that turned into a personal goal to build a Python log parser. That's in development right now! I tried to make it as clean as possible; let me know if you have any questions.

In [1]:
import pandas as pd
import requests

In [2]:
# Set universal variables; directories for inputs & outputs, & the dates of the run you're analyzing
kolmafDir = '/Users/amcguire/Library/Application Support/KoLmafia/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'
kolName   = 'Captain Scotch'
runDates  = ['20190222','20190223','20190224'] 
runNotes  = 'blah blah blah blah /n blah blah blah BLAH /n blah blah, blah blah, Baaaaaah'

In [3]:
# Read in the last parsed run via RunLogSum.ash, for testing purposes
oldRun = pd.read_csv('{}data/{}-runlog_0102.txt'.format(kolmafDir,kolName),sep='\t')

In [4]:
# These are, all things considered, extremely small files. So I just read the whole
#   thing into memory so that it's all accessible without finicky itertools stuff.

# Populate our session dictionary
dDict = {}

for day, session in enumerate(runDates):
    # Python is zero-indexed, so we increment day by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    dDict[day+1] = open('{}sessions/{}_{}.txt'.format(kolmafDir, kolName.replace(" ","_"), session)).read()
    
    # Remove everything before Valhalla & after freeing Ralph
    if day == 0:
        start = dDict[1].find('Welcome to Valhalla!')
        dDict[1] = dDict[1][start:]
        
    else:
        end   = dDict[day+1].find('Freeing King Ralph')
        dDict[day+1] = dDict[day+1][:end]

# Also, to minimize server hits, grab a few files from the Mafia SVN data
mafCombats = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/combats.txt')
mafMonster = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/monsters.txt')
mafMods    = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/modifiers.txt')

In [20]:
mafEncounters = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/encounters.txt')

In [32]:
yolo = {}
for x, line in enumerate(mafEncounters.text.split('\n')):
    try: 
        temp = line.split('\t')
        if len(temp) == 3:
            yolo[x] = {'LOCATION':temp[0], 'ADVTYPE':temp[1], 'NAME':temp[2]}
    except:
        print('error @ {}'.format(x))
        


{1: {'LOCATION': '# Adventure Area Name',
  'ADVTYPE': 'Type',
  'NAME': 'Encounter Title'},
 9: {'LOCATION': 'The Haunted Library',
  'ADVTYPE': 'STOP',
  'NAME': 'History is Fun!'},
 10: {'LOCATION': "Whitey's Grove", 'ADVTYPE': 'STOP', 'NAME': "It's A Sign!"},
 11: {'LOCATION': 'The Haunted Pantry',
  'ADVTYPE': 'STOP',
  'NAME': "The Manor in Which You're Accustomed"},
 12: {'LOCATION': 'The Sleazy Back Alley',
  'ADVTYPE': 'STOP',
  'NAME': 'Under the Knife'},
 13: {'LOCATION': 'The Enormous Greater-Than Sign',
  'ADVTYPE': 'STOP',
  'NAME': 'The Oracle Will See You Now'},
 14: {'LOCATION': 'The Unquiet Garves',
  'ADVTYPE': 'STOP',
  'NAME': 'A Grave Situation'},
 15: {'LOCATION': 'The VERY Unquiet Garves',
  'ADVTYPE': 'STOP',
  'NAME': 'A Grave Situation'},
 16: {'LOCATION': 'Tower Ruins',
  'ADVTYPE': 'STOP',
  'NAME': 'Take a Dusty Look!'},
 17: {'LOCATION': 'Inside the Palindome',
  'ADVTYPE': 'STOP',
  'NAME': 'Drawn Onward'},
 19: {'LOCATION': "Cobb's Knob Laboratory",
  '

In [19]:
findString = '[1]'
buffer     = 6
day        = 1

# simple print w/ buffer for finding things within the logs
for ct, x in enumerate(dDict[day].split(sep='\n')):
    if findString.lower() in x.lower(): 
        print('===========\n'+
              '\n'.join(dDict[day].split(sep='\n')[(ct-buffer):(ct+buffer)])+
              '\n===========\n')

choice.php?pwd&whichchoice=1280&option=2
You acquire an item: FantasyRealm Mage's Hat
You acquire an item: FantasyRealm G. E. M.

equip hat FantasyRealm Mage's Hat

[1] Guano Junction
Encounter: How I Wonder What You're At
You lose 5 hit points
You acquire an item: sonar-in-a-biscuit
You acquire an item: sonar-in-a-biscuit




In [8]:
def monsterLocation(mon):
    ''' Sub-function that uses KOLMafia's combats directory to ascertain
        where a monster is likely to occur (used for establishing wanderer
        likelihood in the absence of full adventure parsing; was used for
        initial tests of digitize capture). Eventually will use this for
        full error-checking of the parser. '''
    
    locs = []
    
    mon = str(mon).strip()
    
    # Read in combats.txt and reformat it via split
    for line in mafCombats.text.split('\n'):
        if   mon in line: 
            locs = locs + [line.split('\t')[0]]
            
        # For some reason, Camel's Toe shows up as "The" Camel's Toe when
        #   you genie wish; this checks for and removes extraneous "The"
        elif mon.lower().replace('the ','') in line.lower():
                locs = locs + [line.split('\t')[0]]
    
    if locs == []: locs = ['FOOTAGE NOT FOUND']
        
    return locs
    
monsterLocation('Green Ops Soldier')

['The Battlefield (Frat Uniform)']

In [11]:
def cleanNames(monName):
    ''' This won't be much of an issue going forward, but
        the Intergnat had an annoying habit of modifying 
        monster names. This attempts to clean that particular 
        detritus to extract pure names. I'm assuming this
        will come back in other forms as well.
        
        Features to add: 
        
            - parse OCRS modifiers 
            - parse "Yes, Can Has" effect'''
    
    # All the strings added to monster names. Haven't added 
    #   OCRS modifiers yet, but intend to. Also would like 
    #   to include rewriting the can has skill, in case it
    #   comes up through the crazy horse, but haven't yet
    
    gnatList = ['NAMED NEIL', 'ELDRITCH HORROR ', 'AND TESLA',
                'WITH BACON!!!', 'WITH SCIENCE']
    
    ocrsMods = []
    
    replList = gnatList + ocrsMods
    
    for repl in replList:
        monName = monName.replace(repl,'')
        
    # Insert renaming for that one cheeseburger skill here
    
    return monName.strip()

In [12]:
def extractWanderers(sessionLogs):
    ''' Quick yet annoying function to attempt to ascertain wanderers. The 
        Digitize parser (unfortunately) doesn't work particularly well yet;
        when ascertaining digitize usage I usually just look over the print &
        mental math for the spreadsheet. The enamorang parser works better,
        which is good, since it's currently the only standard "choice" 
        wanderer as of 1/19. 
        
        Features to add: 
        
            - include detection of vote wanderers 
            - once the XML turn parser is done, revise this to reference it 
            - (you basically fuckin did that already in this hacky POS) '''
    
    wandererDict = {}
    
    # Let's start with digitized monsters!
    
    for day in sessionLogs.keys():
        
        wandererDict[day] = {}
        
        # Tracking digitized monsters is a pain in the ass. Luckily, simply
        #   extracting who they are can be pretty easy with some rough 
        #   nested loops.
        
        digiMons = []
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'casts DIGITIZE!' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                digiMons = digiMons + [monName]
    
        for mon in digiMons:
            
            # Now that we have that list, we want to figure out how many times
            #   they were fought. To do this, we'll need to compare the zone
            #   they originated from to the zones they have been found in.
            #   This method is meh, but kind of works, in the absence of Mafia 
            #   saving intro messages for digitized monsters. 
            
            monLoc = monsterLocation(mon)
            
            # So, cool unintentional thing here; when Mafia logs Witchess, it
            #   totally messes up the turn/combat statement, because it treats
            #   it differently. This is great! It means digitized Witchess
            #   pieces are super easy to find.
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        print('  Could not parse {}'.format(data))
    
        monLOVs = {}
        
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if 'uses the LOV Enamorang' in row:
                x = 0
                
                # Walk backwards through the file w/ a while loop 
                while x < ct:
                    x +=1
                    monName = sessionLogs[day].split('\n')[ct-x]
                    if 'Encounter:' in monName:
                        dataSplit = sessionLogs[day].split('\n')[ct-(x+1)].split('] ')
                        
                        try: 
                            turn = int(dataSplit[0][1:])
                            loct = dataSplit[1]
                        except:
                            turn = -999
                        break
                
                monName = cleanNames(monName.replace('Encounter: ',''))
                monLOVs[turn] = monName
                
        for turn, mon in monLOVs.items():
            
            # Similar process for above, with a few small items. Here, I am
            #   checking turn differential to make positively sure it is the
            #   enamorang'd monster; it has to be 14 turns! 
            
            monLoc = monsterLocation(mon)
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        foundTurn = int(data[0][1:])
                        currLoct = data[1]
                        # For testing
                        #print('Turn {}: fought {} @ {}.'.format(foundTurn,mon,loct))
                    except:
                        # Warn that something was unparsable 
                        print('  Could not parse {}'.format(data))
                    
                    if turn != foundTurn:
                        wandererDict[day][mon] = {'Type':'Enamorang',
                                         'Location':currLoct,
                                         'Turn':foundTurn}
    
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wandererDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wandererDict.keys()],sort=False).T
    
extractWanderers(dDict)

Day #2                                  Day #3  \
            Type                Location Turn       Type   
ghost  Enamorang  genie summoned monster  375  Enamorang   

                                    
                     Location Turn  
ghost  genie summoned monster  549

In [13]:
def extractWishes(sessionLogs):
    ''' Quick function to ascertain wish usage '''
    
    wishDict = {}
    
    for day in sessionLogs.keys():
        
        wishDict[day] = {}
        
        # Wishes aren't quite as easy as pulls, which are easily snagged
        #   on one line with newline split logic, but they're certainly 
        #   easier than digitizations.
        
        # Combat parsing, utilizing the combat to pull monsters out
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "genie summoned monster" in row:
                turn = row[(row.find('[')+1):row.find(']')]
                wishMon = sessionLogs[day].split('\n')[ct+1].replace('Encounter: ','')
                wishMon = cleanNames(wishMon)
                wishDict[day][ct] = {'Type':'Fight',
                                     'Details':wishMon,
                                     'Turn':int(turn)}
        
        # Extra parsing, using the actual wish URL to fill others
        for ct, row in enumerate(sessionLogs[day].split('\n')):
            if "&wish=" in row:
                wishString = row[(row.find('&wish=')+6):]
                try: 
                    
                    # Note we have to use ct+2 due to session logs
                    #   taking 2 lines to get from URL to fight. If
                    #   this ever changes, will need to change.
                    
                    wishDict[day][ct+2]['Type']
                    
                except:
                    
                    wishDict[day][ct] = {'Type':'Non-Fight',
                                         'Details':wishString,
                                         'Turn':'?'}

        # Rename the wishes from line # to wish #
        for i, val in enumerate(wishDict[day]):
            wishDict[day][i+1] = wishDict[day].pop(val)
    
    # Turn this into easily pasted data
    frames = []
    for dayID, d in wishDict.items():
        frames.append(pd.DataFrame.from_dict(d,orient='index').T)
        
    return pd.concat(frames, keys=['Day #{}'.format(x) for x in wishDict.keys()]).T

extractWishes(dDict)

Day #1                              Day #2                            \
           Type                 Details Turn   Type              Details Turn   
1           NaN                     NaN  NaN  Fight  Orcish Frat Boy Spy  117   
2           NaN                     NaN  NaN  Fight         mountain man  199   
3           NaN                     NaN  NaN  Fight                ghost  375   
4     Non-Fight         for+more+wishes    ?    NaN                  NaN  NaN   
5         Fight  Naughty Sorority Nurse   40    NaN                  NaN  NaN   
6     Non-Fight         for+more+wishes    ?    NaN                  NaN  NaN   
7     Non-Fight         for+more+wishes    ?    NaN                  NaN  NaN   
2552      Fight     possessed wine rack   58    NaN                  NaN  NaN   

         Day #3                                        
           Type                          Details Turn  
1         Fight                            ghost  509  
2         Fight                            ghost  549  
3     Non-Fight  to+be+Dragged+Through+the+Coals    ?  
4           NaN                              NaN  NaN  
5           NaN                              NaN  NaN  
6           NaN                              NaN  NaN  
7           NaN                              NaN  NaN  
2552        NaN                              NaN  NaN

In [14]:
def extractPulls(sessionLogs):
    ''' Quick function to extract normal run pulls into a table. '''
    
    pullDict = {}
    
    for day in sessionLogs.keys():
        
        # Find all your pulls; relies on sessions storing them as 'pull: '
        pulls = [i[6:] for i in sessionLogs[day].split('\n') if i.startswith('pull: ')]
        
        # Split out pulls into a 20 item list, for formatting
        numbs = [int(i[0:2].strip()) for i in pulls]
        pullDict[day] = []
        for count, pull in enumerate(pulls):
            pullDict[day] = pullDict[day] + [pull[1:].strip()]*numbs[count]
        
        if len(pullDict[day]) < 20:
            pullDict[day] = pullDict[day] + ['']*(20-len(pullDict[day]))
    
    # Renaming for simplicity/ease of pasting; 
    pullsOut = pd.DataFrame(pullDict).rename(
        index  = lambda x: '#{}'.format(x+1),
        columns= lambda x: 'Day #{}'.format(x))
    
    return pullsOut

extractPulls(dDict)

,Day #1,Day #2,Day #3
#1,,,
#2,,,
#3,,,
#4,,,
#5,,,
#6,,,
#7,,,
#8,,,
#9,,,
#10,,,


In [15]:
def goblinParser(sessionLogs):
    ''' Figure out where/when the user encountered sausage goblins.
        This SHOULD NOT be used as a spading tool, as it does not
        properly track important things like how much meat was in
        the grinder and how many turns the grinder was used. It's
        just for me to think about where I dropped goblins in-run '''
    
    goblinDict = {}
    gobNum = 0
    
    for day in sessionLogs.keys():
        
        # I am keeping this a list in case we eventually find a 
        #   sausage goblin boss...
        
        for mon in ['sausage goblin']:
            
            for ct, row in enumerate(sessionLogs[day].split('\n')):
                if 'Encounter: {}'.format(mon) in row:
                    gobNum = gobNum + 1
                    data = sessionLogs[day].split('\n')[ct-1].split('] ')
                    try:
                        turn = int(data[0][1:])
                        loct = data[1]
                        # print('Turn {}: fought {} @ {}.'.format(turn,mon,loct))
                    except:
                        # Gotta show that error!
                        print('  Could not parse {}'.format(data))
                    
                    goblinDict[gobNum] =      {'Turn': turn, 
                                               'Location': loct,
                                               'Day': day}
    
        
    return pd.DataFrame.from_dict(goblinDict).T
                        
goblinParser(dDict)

,Day,Location,Turn
1,1,The Spooky Forest,2
2,1,The Haunted Library,38
3,1,The Haunted Bedroom,41
4,1,The Haunted Bedroom,43
5,1,The Haunted Bedroom,48
6,1,The Neverending Party,50
7,1,The Neverending Party,50
8,1,The Neverending Party,51
9,2,The Neverending Party,66
10,2,The Neverending Party,67


## Unfinished functions, to be added over time

In [ ]:
def buffSelection(sessionLogs):
    ''' Parse out the IOTM buffs selected in the selected run, by day '''

In [ ]:
def itemSelection(sessionLogs):
    ''' Parse out the IOTM items selected in the selected run, by day '''

In [ ]:
def horseBuffSpading(sessionLogs):
    ''' Find out what buffs you get from the crazy horse, for a simulation
        project I'm working on to establish bounds on crazy horse value '''
